In [2]:
import itertools
import pathlib
import pyam

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as compile

sns.set_style('whitegrid')

<IPython.core.display.Javascript object>

# Read Data

In [4]:
datapath = pathlib.Path('../../data')
fulldf = pyam.IamDataFrame(datapath / 'gidden_brutshin_et_al_2023_data.csv')
fulldf.load_meta(datapath / 'gidden_brutshin_et_al_2023_meta.csv')

pyam - INFO: Running in a notebook, setting up a basic logging at level INFO
pyam.core - INFO: Reading file ..\..\data\gidden_brutshin_et_al_2023_data.csv
pyam.core - INFO: Reading meta indicators


In [5]:
scens = pd.read_excel(datapath / 'scenarios.xlsx', sheet_name='equity_comparison')

Now, we filter out for the necessary scenarios.

In [6]:
equity_data = fulldf.filter(
    scenario=scens['Scenario']
)

# Equity of carbon budgets

First, we need the cumulative debt to be summed up between 1990 and 2019.

In [7]:
equity_data_debt = (
    equity_data
    .filter(
        variable='Emissions|CO2|Debt',
        year=range(1990,2020)
    )
)

In [8]:
for region in equity_data_debt.region:
    variable=f'Cumulative Emissions|CO2|Debt in {region} until 2020'
    data_ts = (
        equity_data_debt
        .filter(region=region)
        .timeseries()
    )
    equity_data.set_meta(
        data_ts.apply(
            lambda x: pyam.cumulative(
                x,
                first_year=1990,
                last_year=2019
            ),
            axis=1
        ),
        name=variable
    )

Identify columns we will use for the analysis of the carbon budgets.

In [9]:
cols_debt = [
    x for x in equity_data.meta 
    if x.startswith('Cumulative Emissions|CO2|Debt')
    and not (x.endswith('until net-zero'))
]

In [10]:
cols_fair = [
    x for x in equity_data.meta
    if x.startswith('Cumulative Emissions|CO2|Fair')
]

In [11]:
cols_model = [
    x for x in equity_data.meta
    if x.startswith('Cumulative Emissions|CO2 in')
]

In [12]:
cols_equity = cols_debt + cols_fair + cols_model

In [13]:
fair_budget_data = equity_data.meta[cols_equity]

Finally, we want to add the debt to the forward-looking fair emission allocations. 

In [14]:
for region in equity_data.region:
    a_to_add = f'Cumulative Emissions|CO2|Fair in {region} until net-zero'
    b_to_add = f'Cumulative Emissions|CO2|Debt in {region} until 2020'
    name = f'Cumulative Emissions|CO2|Fair in {region} until net-zero-including-debt'
    # Add the necessary data
    fair_budget_data.loc[:,name] = (
        fair_budget_data.loc[:,a_to_add] 
        - 
        fair_budget_data.loc[:,b_to_add]
    )

C:\Users\gidden\AppData\Local\Temp\ipykernel_1956\3393361096.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fair_budget_data.loc[:,name] = (
C:\Users\gidden\AppData\Local\Temp\ipykernel_1956\3393361096.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fair_budget_data.loc[:,name] = (
C:\Users\gidden\AppData\Local\Temp\ipykernel_1956\3393361096.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

Now we need to reshape the data so that we can plot it out.

In [15]:
fair_budget_reshaped = (
    fair_budget_data
    .stack()
    .reset_index(level=-1)
    .rename(
        columns={0:'value'}
    )
)

In [16]:
fair_budget_reshaped['level_2'] = fair_budget_reshaped['level_2'].apply(
    lambda x: x.replace('Annex I', 'Developed')
)

In [17]:
split_elements = (
    fair_budget_reshaped['level_2']
    .apply(lambda x: x.split(' '))
)

In [18]:
fair_budget_reshaped.loc[:,'variable'] = split_elements.apply(lambda x: x[1])
fair_budget_reshaped.loc[:,'region'] = split_elements.apply(lambda x: x[3])
fair_budget_reshaped.loc[:,'consideration'] = split_elements.apply(lambda x: x[5])

In [19]:
fair_budget_reshaped = (
    fair_budget_reshaped[
        fair_budget_reshaped['variable']
        .isin(['Emissions|CO2|Fair','Emissions|CO2'])
    ]
)

In [20]:
fair_budget_reshaped

level_2  \
model             scenario                                                                                        
GENIE_SSP2_v4.1.7 EN_NPi2020-DACm-MP-median-stor3-final_1000  Cumulative Emissions|CO2|Fair in AFR until net...   
                  EN_NPi2020-DACm-MP-median-stor3-final_1000  Cumulative Emissions|CO2|Fair in Africa until ...   
                  EN_NPi2020-DACm-MP-median-stor3-final_1000  Cumulative Emissions|CO2|Fair in Developed unt...   
                  EN_NPi2020-DACm-MP-median-stor3-final_1000  Cumulative Emissions|CO2|Fair in Asia until ne...   
                  EN_NPi2020-DACm-MP-median-stor3-final_1000  Cumulative Emissions|CO2|Fair in CPA until net...   
...                                                                                                         ...   
                  EN_NPi2020-stor3-final_700                  Cumulative Emissions|CO2|Fair in PAO until net...   
                  EN_NPi2020-stor3-final_700                  Cumulative Emissions|CO2|Fair in PAS until net...   
                  EN_NPi2020-stor3-final_700                  Cumulative Emissions|CO2|Fair in SAS until net...   
                  EN_NPi2020-stor3-final_700                  Cumulative Emissions|CO2|Fair in WEU until net...   
                  EN_NPi2020-stor3-final_700                  Cumulative Emissions|CO2|Fair in World until n...   

                                                                      value  \
model             scenario                                                    
GENIE_SSP2_v4.1.7 EN_NPi2020-DACm-MP-median-stor3-final_1000  149230.368585   
                  EN_NPi2020-DACm-MP-median-stor3-final_1000  216599.511197   
                  EN_NPi2020-DACm-MP-median-stor3-final_1000  168181.019392   
                  EN_NPi2020-DACm-MP-median-stor3-final_1000  310442.610053   
                  EN_NPi2020-DACm-MP-median-stor3-final_1000  165846.347047   
...                                                                     ...   
                  EN_NPi2020-stor3-final_700                  -16130.156724   
                  EN_NPi2020-stor3-final_700                   43150.972070   
                  EN_NPi2020-stor3-final_700                  360949.325563   
                  EN_NPi2020-stor3-final_700                    3798.360654   
                  EN_NPi2020-stor3-final_700                  640603.766001   

                                                                        variable  \
model             scenario                                                         
GENIE_SSP2_v4.1.7 EN_NPi2020-DACm-MP-median-stor3-final_1000  Emissions|CO2|Fair   
                  EN_NPi2020-DACm-MP-median-stor3-final_1000  Emissions|CO2|Fair   
                  EN_NPi2020-DACm-MP-median-stor3-final_1000  Emissions|CO2|Fair   
                  EN_NPi2020-DACm-MP-median-stor3-final_1000  Emissions|CO2|Fair   
                  EN_NPi2020-DACm-MP-median-stor3-final_1000  Emissions|CO2|Fair   
...                                                                          ...   
                  EN_NPi2020-stor3-final_700                  Emissions|CO2|Fair   
                  EN_NPi2020-stor3-final_700                  Emissions|CO2|Fair   
                  EN_NPi2020-stor3-final_700                  Emissions|CO2|Fair   
                  EN_NPi2020-stor3-final_700                  Emissions|CO2|Fair   
                  EN_NPi2020-stor3-final_700                  Emissions|CO2|Fair   

                                                                 region  \
model             scenario                                                
GENIE_SSP2_v4.1.7 EN_NPi2020-DACm-MP-median-stor3-final_1000        AFR   
                  EN_NPi2020-DACm-MP-median-stor3-final_1000     Africa   
                  EN_NPi2020-DACm-MP-median-stor3-final_1000  Developed   
                  EN_NPi2020-DACm-MP-median-stor3-final_1000       Asia   
                  EN_NPi2020-DACm-MP-

Copy the model data and assign the additional consideration type so that we can pivot the data.

In [21]:
fair_budget_reshaped_model = fair_budget_reshaped[fair_budget_reshaped['variable']=='Emissions|CO2']

In [22]:
fair_budget_reshaped_model['consideration'] = 'net-zero-including-debt'

C:\Users\gidden\AppData\Local\Temp\ipykernel_1956\689366091.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fair_budget_reshaped_model['consideration'] = 'net-zero-including-debt'


In [23]:
fair_budget_reshaped = pd.concat(
    [fair_budget_reshaped, fair_budget_reshaped_model]
)

In [24]:
fair_budget_final = pd.pivot_table(
    fair_budget_reshaped.drop(columns='level_2').reset_index(),
    index=['model','scenario','region','consideration'],
    columns=['variable'],
    values='value'
)

In [25]:
fair_budget_final.index.get_level_values('region').unique()

Index(['AFR', 'Africa', 'Asia', 'CPA', 'China', 'Developed', 'EEU', 'FSU',
       'LAM', 'Latin', 'MEA', 'NAM', 'PAO', 'PAS', 'SAS', 'WEU', 'World'],
      dtype='object', name='region')

In [26]:
regions_to_assess = ['Developed','LAM','Africa','China','Asia']

In [27]:
fair_budget_final.head()

variable                                                                                     Emissions|CO2  \
model             scenario                                   region consideration                            
GENIE_SSP2_v4.1.7 EN_NPi2020-DACm-MP-median-stor3-final_1000 AFR    net-zero                  39526.563765   
                                                                    net-zero-including-debt   39526.563765   
                                                             Africa net-zero                 168342.207398   
                                                                    net-zero-including-debt  168342.207398   
                                                             Asia   net-zero                 234390.778236   

variable                                                                                     Emissions|CO2|Fair  
model             scenario                                   region consideration                                
GENIE_SSP2_v4.1.7 EN_NPi2020-DACm-MP-median-stor3-final_1000 AFR    net-zero                      149230.368585  
                                                                    net-zero-including-debt       203586.947442  
                                                             Africa net-zero                      216599.511197  
                                                                    net-zero-including-debt       274798.539020  
                                                             Asia   net-zero                      310442.610053

In [28]:
fair_budget_final = fair_budget_final.loc[pd.IndexSlice[:, :, regions_to_assess,:]]

Now, we take care of unit conversion issues.

In [29]:
fair_budget_final /= 1e3

In [30]:
fair_budget_final = fair_budget_final.assign(unit='Gt CO2')

In [31]:
fair_budget_final

variable                                                                                        Emissions|CO2  \
model             scenario                                   region    consideration                            
GENIE_SSP2_v4.1.7 EN_NPi2020-DACm-MP-median-stor3-final_1000 Developed net-zero                    250.817605   
                                                                       net-zero-including-debt     250.817605   
                  EN_NPi2020-DACm-MP-median-stor3-final_500  Developed net-zero                    146.873328   
                                                                       net-zero-including-debt     146.873328   
                  EN_NPi2020-DACm-MP-median-stor3-final_700  Developed net-zero                    195.612380   
...                                                                                                       ...   
                  EN_NPi2020-stor3-final_1000                Asia      net-zero-including-debt     224.514720   
                  EN_NPi2020-stor3-final_500                 Asia      net-zero                     91.447877   
                                                                       net-zero-including-debt      91.447877   
                  EN_NPi2020-stor3-final_700                 Asia      net-zero                    134.839168   
                                                                       net-zero-including-debt     134.839168   

variable                                                                                        Emissions|CO2|Fair  \
model             scenario                                   region    consideration                                 
GENIE_SSP2_v4.1.7 EN_NPi2020-DACm-MP-median-stor3-final_1000 Developed net-zero                         168.181019   
                                                                       net-zero-including-debt          -89.453671   
                  EN_NPi2020-DACm-MP-median-stor3-final_500  Developed net-zero                          82.141269   
                                                                       net-zero-including-debt         -175.493421   
                  EN_NPi2020-DACm-MP-median-stor3-final_700  Developed net-zero                         116.356526   
...                                                                                                            ...   
                  EN_NPi2020-stor3-final_1000                Asia      net-zero-including-debt          505.571042   
                  EN_NPi2020-stor3-final_500                 Asia      net-zero                         146.988450   
                                                                       net-zero-including-debt          339.824065   
                  EN_NPi2020-stor3-final_700                 Asia      net-zero                         211.264683   
                                                                       net-zero-including-debt          404.100298   

variable                                                                                          unit  
model             scenario                                   region    consideration                    
GENIE_SSP2_v4.1.7 EN_NPi2020-DACm-MP-median-stor3-final_1000 Developed net-zero                 Gt CO2  
                                                                       net-zero-including-debt  Gt CO2  
                  EN_NPi2020-DACm-MP-median-stor3-final_500  Developed net-zero                 Gt CO2  
                                                                       net-zero-including-debt  Gt CO2  
                  EN_NPi2020-DACm-MP-median-stor3-final_700  Developed net-zero                 Gt CO2  
...                                                                                                ...  
                  EN_NPi2020-stor3-final_1000                Asia      net-zero-including-debt  Gt CO2  
                  EN_NPi2020-stor3-final_500      

In [32]:
fig5_col1_data = (
    pd.merge(
        fair_budget_final.reset_index(),
        scens, 
        right_on=['Scenario'], 
        left_on=['scenario']
    )
    .drop(columns=['Model','Scenario'])
)

In [33]:
fig5_col1_data.head()

,model,scenario,region,consideration,Emissions|CO2,Emissions|CO2|Fair,unit,Temperature,DAC,Technoeconomic,Diffusion,Governance,Governance SSP
0,GENIE_SSP2_v4.1.7,EN_NPi2020-DACm-MP-median-stor3-final_1000,Developed,net-zero,250.817605,168.181019,Gt CO2,C3 with DAC,DAC Included,Medium,Medium,Unconstrained Governance,NaN
1,GENIE_SSP2_v4.1.7,EN_NPi2020-DACm-MP-median-stor3-final_1000,Developed,net-zero-including-debt,250.817605,-89.453671,Gt CO2,C3 with DAC,DAC Included,Medium,Medium,Unconstrained Governance,NaN
2,GENIE_SSP2_v4.1.7,EN_NPi2020-DACm-MP-median-stor3-final_1000,LAM,net-zero,30.495994,77.750303,Gt CO2,C3 with DAC,DAC Included,Medium,Medium,Unconstrained Governance,NaN
3,GENIE_SSP2_v4.1.7,EN_NPi2020-DACm-MP-median-stor3-final_1000,LAM,net-zero-including-debt,30.495994,52.915034,Gt CO2,C3 with DAC,DAC Included,Medium,Medium,Unconstrained Governance,NaN
4,GENIE_SSP2_v4.1.7,EN_NPi2020-DACm-MP-median-stor3-final_1000,Africa,net-zero,168.342207,216.599511,Gt CO2,C3 with DAC,DAC Included,Medium,Medium,Unconstrained Governance,NaN


Calculate the deviation from the fair benchmarks.

In [34]:
fig5_col1_data.loc[:,'delta'] = fig5_col1_data['Emissions|CO2|Fair'] - fig5_col1_data['Emissions|CO2']

In [35]:
fig5_col1_data

,model,scenario,region,consideration,Emissions|CO2,Emissions|CO2|Fair,unit,Temperature,DAC,Technoeconomic,Diffusion,Governance,Governance SSP,delta
0,GENIE_SSP2_v4.1.7,EN_NPi2020-DACm-MP-median-stor3-final_1000,Developed,net-zero,250.817605,168.181019,Gt CO2,C3 with DAC,DAC Included,Medium,Medium,Unconstrained Governance,NaN,-82.636585
1,GENIE_SSP2_v4.1.7,EN_NPi2020-DACm-MP-median-stor3-final_1000,Developed,net-zero-including-debt,250.817605,-89.453671,Gt CO2,C3 with DAC,DAC Included,Medium,Medium,Unconstrained Governance,NaN,-340.271276
2,GENIE_SSP2_v4.1.7,EN_NPi2020-DACm-MP-median-stor3-final_1000,LAM,net-zero,30.495994,77.750303,Gt CO2,C3 with DAC,DAC Included,Medium,Medium,Unconstrained Governance,NaN,47.254308
3,GENIE_SSP2_v4.1.7,EN_NPi2020-DACm-MP-median-stor3-final_1000,LAM,net-zero-including-debt,30.495994,52.915034,Gt CO2,C3 with DAC,DAC Included,Medium,Medium,Unconstrained Governance,NaN,22.419039
4,GENIE_SSP2_v4.1.7,EN_NPi2020-DACm-MP-median-stor3-final_1000,Africa,net-zero,168.342207,216.599511,Gt CO2,C3 with DAC,DAC Included,Medium,Medium,Unconstrained Governance,NaN,48.257304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,GENIE_SSP2_v4.1.7,EN_NPi2020-stor3-final_700,Africa,net-zero-including-debt,94.707209,201.247441,Gt CO2,C2 without DAC,No DAC,No DAC,No DAC,Unconstrained Governance,NaN,106.540232
86,GENIE_SSP2_v4.1.7,EN_NPi2020-stor3-final_700,China,net-zero,151.872116,116.699600,Gt CO2,C2 without DAC,No DAC,No DAC,No DAC,Unconstrained Governance,NaN,-35.172516
87,GENIE_SSP2_v4.1.7,EN_NPi2020-stor3-final_700,China,net-zero-including-debt,151.872116,161.913741,Gt CO2,C2 without DAC,No DAC,No DAC,No DAC,Unconstrained Governance,NaN,10.041625
88,GENIE_SSP2_v4.1.7,EN_NPi2020-stor3-final_700,Asia,net-zero,134.839168,211.264683,Gt CO2,C2 without DAC,No DAC,No DAC,No DAC,Unconstrained Governance,NaN,76.425515


Save this out for plot generation.

In [36]:
fig5_col1_data.to_excel(
    'fig5_col1_data.xlsx'
)